# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import requests
import gmaps
import hvplot.pandas

# Import API key
from api_keys import geoapify_key
from api_keys import gmaps_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mweka,-4.8500,21.5667,66.04,97,64,0.58,CD,1667415704
1,1,ponta do sol,32.6667,-17.1000,73.00,89,99,1.79,PT,1667415705
2,2,puerto ayora,-0.7393,-90.3518,75.16,73,59,10.92,EC,1667415597
3,3,sztum,53.9208,19.0307,50.32,83,4,7.74,PL,1667415705
4,4,hermanus,-34.4187,19.2345,65.80,77,1,10.65,ZA,1667415590
5,5,ushuaia,-54.8000,-68.3000,56.82,54,40,5.75,AR,1667415524
6,6,xichang,27.8964,102.2634,54.01,87,2,2.15,CN,1667415706
7,7,touros,-5.1989,-35.4608,82.65,61,81,13.11,BR,1667415706
8,8,eyrarbakki,63.8630,-21.1481,39.20,81,100,13.67,IS,1667415707
9,9,bethel,41.3712,-73.4140,71.96,46,0,1.99,US,1667415707


In [8]:
# Configure the map plot
gmaps.configure(api_key=gmaps_key)

# YOUR CODE HERE
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"].astype(float)

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=80, point_radius=1)

fig.add_layer(heatmap_layer)

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp = (city_data_df["Max Temp"] <= 75) & (city_data_df["Max Temp"] > 65)
wind_speed = city_data_df["Wind Speed"] < 5
cloudiness = city_data_df["Cloudiness"] <= 30

vac_cities_df = city_data_df[max_temp & wind_speed & cloudiness]

# Drop any rows with null values
vac_cities_df.dropna()

# Display sample data
vac_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,bethel,41.3712,-73.4140,71.96,46,0,1.99,US,1667415707
16,16,santa vitoria do palmar,-33.5189,-53.3681,66.52,40,0,4.61,BR,1667415711
34,34,albany,42.6001,-73.9662,66.58,50,9,1.01,US,1667415368
42,42,muroto,33.2833,134.1500,67.06,74,0,2.98,JP,1667415718
54,54,jamestown,42.0970,-79.2353,67.01,53,0,3.00,US,1667415587
149,149,paso de los toros,-32.8167,-56.5167,74.25,22,0,4.23,UY,1667415776
196,196,bowen,-20.0167,148.2333,70.81,61,0,4.92,AU,1667415798
287,287,palmer,42.1584,-72.3287,71.20,39,10,1.99,US,1667415829
302,302,roma,43.2128,-75.4557,65.73,52,0,3.44,US,1667415833
321,321,mareeba,-17.0000,145.4333,69.44,83,0,2.30,AU,1667415839


In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(vac_cities_df, columns = ["City", "Country", "Lat","Lng","Humidity"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df.head(1)

,City,Country,Lat,Lng,Humidity,Hotel Name
9,bethel,US,41.3712,-73.414,46,


In [11]:
# Set parameters to search for a hotel
radius = 5000
params = {"categories":"accommodation.hotel",
          "apiKey":geoapify_key,
          "limit":5,}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =f'circle:{lng},{lat},{radius}'
    params["bias"] =f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = f"No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

#dropping hotels with no Hotels 
hotel_df.drop(hotel_df.loc[hotel_df['Hotel Name'] == 'No hotel found'].index, inplace=True)    

hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
santa vitoria do palmar - nearest hotel: Hotel Brasil
albany - nearest hotel: No hotel found
muroto - nearest hotel: 民宿うらしま
jamestown - nearest hotel: DoubleTree Jamestown
paso de los toros - nearest hotel: Hotel Italiana
bowen - nearest hotel: Whitsunday Sands Resort
palmer - nearest hotel: No hotel found
roma - nearest hotel: Wingate by Wyndham Rome
mareeba - nearest hotel: The Gateway Hotel
gat - nearest hotel: No hotel found
constantine - nearest hotel: Hôtel El Hawa Talk
camacha - nearest hotel: Praia Dourada
ingham - nearest hotel: No hotel found
port-de-bouc - nearest hotel: B&B Hôtel
amli - nearest hotel: Lotus Kamat Resort
durango - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
9,bethel,US,41.3712,-73.4140,46,Hampton Inn Danbury
16,santa vitoria do palmar,BR,-33.5189,-53.3681,40,Hotel Brasil
42,muroto,JP,33.2833,134.1500,74,民宿うらしま
54,jamestown,US,42.0970,-79.2353,53,DoubleTree Jamestown
149,paso de los toros,UY,-32.8167,-56.5167,22,Hotel Italiana
196,bowen,AU,-20.0167,148.2333,61,Whitsunday Sands Resort
302,roma,US,43.2128,-75.4557,52,Wingate by Wyndham Rome
321,mareeba,AU,-17.0000,145.4333,83,The Gateway Hotel
442,constantine,DZ,36.3650,6.6147,30,Hôtel El Hawa Talk
473,camacha,PT,33.0833,-16.3333,78,Praia Dourada


In [12]:
#add hotel names 
hotel_list = hotel_df["Hotel Name"].tolist()

#add the hotel's coordinates
locations = hotel_df[["Lat", "Lng"]]

#plot the hotels with info box
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel Name: {hotel}" for hotel in hotel_list])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))